## Naive and Aware Times – Advanced Exercises

This notebook contains a series of **advanced (but not too advanced)** exercises focusing on:

- Naive vs aware `datetime` objects
- `tzinfo`, `timezone`, and UTC offsets
- Converting between local times and UTC
- Parsing ISO 8601 strings with offsets

Each exercise has:
- A **problem description** (markdown)
- A **solution** (code), written using clean, best-practice Python


---
### Exercise 1 – Detecting Naive vs Aware `datetime`

**Goal:** Implement helper functions to check whether a `datetime` is naive or aware.

**Requirements:**
1. Implement two functions:
   ```python
   def is_aware(dt: datetime.datetime) -> bool: ...
   def is_naive(dt: datetime.datetime) -> bool: ...
   ```
2. Use the definitions from the standard library docs:
   - A datetime is **naive** if `dt.tzinfo is None` or `dt.tzinfo.utcoffset(dt) is None`.
   - Otherwise, it is **aware**.
3. If the input is not a `datetime.datetime` instance, raise a `TypeError` with a clear message.
4. Demonstrate the functions with:
   - A naive datetime created with `datetime(2020, 1, 1, 12, 0)`.
   - An aware UTC datetime using `tzinfo=timezone.utc`.
   - An aware fixed-offset datetime created with `timezone(timedelta(hours=-4), 'EDT')`.

This exercise practices:
- Understanding the official definition of naive vs aware
- Using `tzinfo.utcoffset(...)`
- Input validation and helpful error messages


In [1]:
from __future__ import annotations
import datetime as _dt
from datetime import timezone, timedelta


def is_aware(dt: _dt.datetime) -> bool:
    """Return True if `dt` is timezone-aware, False otherwise.

    A datetime is aware if tzinfo is not None and utcoffset is not None.
    """
    if not isinstance(dt, _dt.datetime):
        raise TypeError(f"Expected datetime, got {type(dt)!r}")
    return dt.tzinfo is not None and dt.tzinfo.utcoffset(dt) is not None


def is_naive(dt: _dt.datetime) -> bool:
    """Return True if `dt` is naive, False otherwise.

    A datetime is naive if it is not aware.
    """
    return not is_aware(dt)


if __name__ == "__main__":
    naive = _dt.datetime(2020, 1, 1, 12, 0)
    aware_utc = _dt.datetime(2020, 1, 1, 12, 0, tzinfo=timezone.utc)
    tz_edt = timezone(timedelta(hours=-4), "EDT")
    aware_edt = _dt.datetime(2020, 1, 1, 12, 0, tzinfo=tz_edt)

    for label, value in [
        ("naive", naive),
        ("aware_utc", aware_utc),
        ("aware_edt", aware_edt),
    ]:
        print(f"{label}: {value!r}")
        print("  is_aware:", is_aware(value))
        print("  is_naive:", is_naive(value))
        print("-")


naive: datetime.datetime(2020, 1, 1, 12, 0)
  is_aware: False
  is_naive: True
-
aware_utc: datetime.datetime(2020, 1, 1, 12, 0, tzinfo=datetime.timezone.utc)
  is_aware: True
  is_naive: False
-
aware_edt: datetime.datetime(2020, 1, 1, 12, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000), 'EDT'))
  is_aware: True
  is_naive: False
-


---
### Exercise 2 – Converting Aware Datetimes to Naive UTC

**Goal:** Implement a safe conversion from an aware datetime (any fixed offset) to a **naive UTC** datetime.

**Requirements:**
1. Implement a function:
   ```python
   def to_utc_naive(dt: datetime.datetime) -> datetime.datetime:
       ...
   ```
2. Behavior:
   - If `dt` is naive, raise `ValueError`: the function expects an aware datetime.
   - Convert `dt` to UTC using `.astimezone(timezone.utc)`.
   - Return a new datetime with identical year/month/day/hour/minute/second/microsecond
     but with `tzinfo=None`.
3. Demonstrate with:
   - A `-4` hour offset (EDT-style).
   - A `+2` hour offset.

This exercise practices:
- Using `.astimezone()` correctly
- Dropping timezone information via `.replace(tzinfo=None)`
- Being explicit that naive UTC is your internal representation


In [2]:
import datetime as _dt
from datetime import timezone, timedelta


def to_utc_naive(dt: _dt.datetime) -> _dt.datetime:
    """Convert an aware datetime to a naive UTC datetime.

    Raises ValueError if `dt` is naive.
    """
    if not isinstance(dt, _dt.datetime):
        raise TypeError(f"Expected datetime, got {type(dt)!r}")

    if dt.tzinfo is None or dt.tzinfo.utcoffset(dt) is None:
        raise ValueError("to_utc_naive expects an aware datetime")

    dt_utc = dt.astimezone(timezone.utc)
    return dt_utc.replace(tzinfo=None)


if __name__ == "__main__":
    tz_minus_4 = timezone(timedelta(hours=-4), "UTC-4")
    tz_plus_2 = timezone(timedelta(hours=2), "UTC+2")

    dt_minus_4 = _dt.datetime(2020, 5, 15, 13, 30, tzinfo=tz_minus_4)
    dt_plus_2 = _dt.datetime(2020, 5, 15, 13, 30, tzinfo=tz_plus_2)

    for label, value in [
        ("UTC-4", dt_minus_4),
        ("UTC+2", dt_plus_2),
    ]:
        print(label, "aware:", value, "tzinfo=", value.tzinfo)
        naive_utc = to_utc_naive(value)
        print("  -> naive UTC:", naive_utc, "tzinfo=", naive_utc.tzinfo)
        print("-")


UTC-4 aware: 2020-05-15 13:30:00-04:00 tzinfo= UTC-4
  -> naive UTC: 2020-05-15 17:30:00 tzinfo= None
-
UTC+2 aware: 2020-05-15 13:30:00+02:00 tzinfo= UTC+2
  -> naive UTC: 2020-05-15 11:30:00 tzinfo= None
-


---
### Exercise 3 – Treating Naive Datetimes as UTC and Converting to Local Time

**Goal:** Convert a naive datetime that represents **UTC** into an aware datetime in a fixed-offset timezone.

**Requirements:**
1. Implement a function:
   ```python
   def utc_naive_to_tz(
       dt_naive: datetime.datetime,
       *,
       offset_hours: int,
       name: str = "",
   ) -> datetime.datetime:
       ...
   ```
2. Behavior:
   - `dt_naive` **must** be naive; raise `ValueError` if it is aware.
   - Treat `dt_naive` as a UTC datetime.
   - Build a timezone using `timezone(timedelta(hours=offset_hours), name)`.
   - Return a datetime in that timezone with `.astimezone(...)`.
3. Demonstrate with:
   - A UTC naive datetime converted to UTC-4 (EDT-style).
   - The same UTC naive datetime converted to UTC+9.

This exercise practices:
- Adding `tzinfo` to naive datetimes
- Using fixed-offset timezones
- Keeping the "treat-naive-as-UTC" convention explicit


In [3]:
import datetime as _dt
from datetime import timezone, timedelta


def utc_naive_to_tz(
    dt_naive: _dt.datetime,
    *,
    offset_hours: int,
    name: str = "",
) -> _dt.datetime:
    """Treat a naive datetime as UTC and convert to a fixed-offset timezone.

    Parameters
    ----------
    dt_naive : datetime.datetime
        Naive datetime representing UTC.
    offset_hours : int
        Offset from UTC in hours (e.g. -4 for UTC-4).
    name : str, optional
        Name of the timezone, e.g. 'EDT'.
    """
    if not isinstance(dt_naive, _dt.datetime):
        raise TypeError(f"Expected datetime, got {type(dt_naive)!r}")

    if dt_naive.tzinfo is not None:
        raise ValueError("utc_naive_to_tz expects a naive datetime (tzinfo is None)")

    tz = timezone(timedelta(hours=offset_hours), name or f"UTC{offset_hours:+d}")
    dt_utc = dt_naive.replace(tzinfo=timezone.utc)
    return dt_utc.astimezone(tz)


if __name__ == "__main__":
    dt_naive = _dt.datetime(2020, 5, 15, 17, 30)
    print("Naive UTC:", dt_naive, "tzinfo=", dt_naive.tzinfo)

    edt = utc_naive_to_tz(dt_naive, offset_hours=-4, name="EDT")
    jst = utc_naive_to_tz(dt_naive, offset_hours=9, name="JST")

    print("-> EDT:", edt, "tzinfo=", edt.tzinfo)
    print("-> JST:", jst, "tzinfo=", jst.tzinfo)


Naive UTC: 2020-05-15 17:30:00 tzinfo= None
-> EDT: 2020-05-15 13:30:00-04:00 tzinfo= EDT
-> JST: 2020-05-16 02:30:00+09:00 tzinfo= JST


---
### Exercise 4 – Parsing ISO 8601 Strings with Offsets into Naive UTC

**Goal:** Parse ISO 8601 datetime strings that may include offsets or `Z`, and normalize them into naive UTC datetimes.

**Requirements:**
1. Implement a function:
   ```python
   def parse_iso_to_utc_naive(iso_str: str, *, assume_naive_utc: bool = True) -> datetime.datetime:
       ...
   ```
2. Behavior:
   - Accept strings like:
     - `"2020-05-15T13:30:00-04:00"`
     - `"2020-05-15T17:30:00Z"`
     - `"2020-05-15T17:30:00"` (no offset)
   - If the string ends with `Z`, replace it with `+00:00` before calling `datetime.fromisoformat`.
   - If the parsed datetime is aware:
     - Convert to UTC with `.astimezone(timezone.utc)` and drop `tzinfo`.
   - If the parsed datetime is naive:
     - If `assume_naive_utc` is True, treat it as UTC and return as-is.
     - Otherwise, raise `ValueError` (require explicit offset).
3. Demonstrate with:
   - An offset string (`-04:00`).
   - A `Z`-terminated string.
   - A naive string with both `assume_naive_utc=True` and `False`.

This exercise practices:
- Robust ISO 8601 parsing
- Normalizing various representations to a single internal form (naive UTC)
- Configurable assumptions about naive datetimes


In [4]:
import datetime as _dt
from datetime import timezone


def parse_iso_to_utc_naive(
    iso_str: str,
    *,
    assume_naive_utc: bool = True,
) -> _dt.datetime:
    """Parse an ISO 8601 datetime string into a naive UTC datetime.

    Handles strings with offsets, 'Z', or no offset. The behavior for
    naive inputs is controlled by `assume_naive_utc`.
    """
    if iso_str.endswith("Z"):
        iso_str = iso_str[:-1] + "+00:00"

    try:
        dt = _dt.datetime.fromisoformat(iso_str)
    except ValueError as exc:
        raise ValueError(f"Invalid ISO datetime string: {iso_str!r}") from exc

    if dt.tzinfo is None:
        if assume_naive_utc:
            # Treat as already UTC naive
            return dt
        raise ValueError(
            "Parsed datetime is naive but assume_naive_utc=False. "
            "Please provide an explicit offset or enable assume_naive_utc."
        )

    # Aware datetime: normalize to UTC and drop tzinfo
    dt_utc = dt.astimezone(timezone.utc)
    return dt_utc.replace(tzinfo=None)


if __name__ == "__main__":
    examples = [
        "2020-05-15T13:30:00-04:00",  # with offset
        "2020-05-15T17:30:00Z",       # Z suffix
        "2020-05-15T17:30:00",        # naive
    ]

    print("assume_naive_utc=True (default):")
    for s in examples:
        try:
            result = parse_iso_to_utc_naive(s)
            print(f"  {s!r} -> {result!r} (tzinfo={result.tzinfo})")
        except ValueError as exc:
            print(f"  {s!r} -> Error: {exc}")

    print("\nassume_naive_utc=False:")
    for s in examples:
        try:
            result = parse_iso_to_utc_naive(s, assume_naive_utc=False)
            print(f"  {s!r} -> {result!r} (tzinfo={result.tzinfo})")
        except ValueError as exc:
            print(f"  {s!r} -> Error: {exc}")


assume_naive_utc=True (default):
  '2020-05-15T13:30:00-04:00' -> datetime.datetime(2020, 5, 15, 17, 30) (tzinfo=None)
  '2020-05-15T17:30:00Z' -> datetime.datetime(2020, 5, 15, 17, 30) (tzinfo=None)
  '2020-05-15T17:30:00' -> datetime.datetime(2020, 5, 15, 17, 30) (tzinfo=None)

assume_naive_utc=False:
  '2020-05-15T13:30:00-04:00' -> datetime.datetime(2020, 5, 15, 17, 30) (tzinfo=None)
  '2020-05-15T17:30:00Z' -> datetime.datetime(2020, 5, 15, 17, 30) (tzinfo=None)
  '2020-05-15T17:30:00' -> Error: Parsed datetime is naive but assume_naive_utc=False. Please provide an explicit offset or enable assume_naive_utc.


---
### Exercise 5 – Normalizing a Mixed List of Datetimes to Naive UTC

**Goal:** Given a list of datetimes (some naive, some aware), normalize them all to **naive UTC** using a consistent policy.

**Requirements:**
1. Implement a function:
   ```python
   def normalize_datetimes_to_utc_naive(
       values: list[datetime.datetime],
       *,
       assume_naive_utc: bool = True,
   ) -> list[datetime.datetime]:
       ...
   ```
2. For each element in `values`:
   - If it is aware, convert to UTC and drop `tzinfo`.
   - If it is naive and `assume_naive_utc` is True, leave it as-is.
   - If it is naive and `assume_naive_utc` is False, raise `ValueError`.
3. Validate all elements are `datetime.datetime`; otherwise raise `TypeError`.
4. Demonstrate with a list containing:
   - A naive UTC datetime.
   - An aware UTC datetime.
   - An aware UTC-5 datetime.
   - Show behavior for `assume_naive_utc=True` and `False`.

This exercise practices:
- Applying your naive/aware logic consistently across a collection
- Designing clear normalization policies
- Building on previously implemented primitives in a clean way


In [5]:
import datetime as _dt
from datetime import timezone, timedelta
from typing import Iterable, List


def normalize_datetimes_to_utc_naive(
    values: Iterable[_dt.datetime],
    *,
    assume_naive_utc: bool = True,
) -> List[_dt.datetime]:
    """Normalize a collection of datetimes to naive UTC.

    - Aware datetimes are converted to UTC and tzinfo is removed.
    - Naive datetimes are either kept as-is (assumed UTC) or rejected
      depending on `assume_naive_utc`.
    """
    normalized: List[_dt.datetime] = []

    for dt in values:
        if not isinstance(dt, _dt.datetime):
            raise TypeError(f"Expected datetime, got {type(dt)!r}")

        if dt.tzinfo is None or dt.tzinfo.utcoffset(dt) is None:
            # naive
            if assume_naive_utc:
                normalized.append(dt)
            else:
                raise ValueError(
                    f"Naive datetime encountered: {dt!r}. "
                    "Set assume_naive_utc=True to treat as UTC."
                )
        else:
            # aware: convert to UTC and drop tzinfo
            dt_utc = dt.astimezone(timezone.utc)
            normalized.append(dt_utc.replace(tzinfo=None))

    return normalized


if __name__ == "__main__":
    tz_utc_minus_5 = timezone(timedelta(hours=-5), "UTC-5")
    tz_utc = timezone.utc

    naive_utc = _dt.datetime(2020, 5, 15, 17, 30)
    aware_utc = _dt.datetime(2020, 5, 15, 17, 30, tzinfo=tz_utc)
    aware_minus_5 = _dt.datetime(2020, 5, 15, 12, 30, tzinfo=tz_utc_minus_5)

    values = [naive_utc, aware_utc, aware_minus_5]

    print("assume_naive_utc=True:")
    for dt in normalize_datetimes_to_utc_naive(values, assume_naive_utc=True):
        print("  ", dt, "tzinfo=", dt.tzinfo)

    print("\nassume_naive_utc=False:")
    try:
        normalize_datetimes_to_utc_naive(values, assume_naive_utc=False)
    except ValueError as exc:
        print("  Error:", exc)


assume_naive_utc=True:
   2020-05-15 17:30:00 tzinfo= None
   2020-05-15 17:30:00 tzinfo= None
   2020-05-15 17:30:00 tzinfo= None

assume_naive_utc=False:
  Error: Naive datetime encountered: datetime.datetime(2020, 5, 15, 17, 30). Set assume_naive_utc=True to treat as UTC.
